### 1. Import librarries

In [1]:
import emoji
from underthesea import word_tokenize, sent_tokenize
from viet_text_tools import normalize_diacritics
import re 
import pandas as pd

import string
import os

### 2. Data Loading

In [2]:
file_names = ["reviews_1789.csv", "reviews_1815.csv", "reviews_1846.csv"]

dir = 'E:\MY CAREER\Internship Project\EDA\dataset\\tiki'
dataframes = []
for file in file_names:
    path = os.path.join(dir, file)
    try:
        df = pd.read_csv(path)
        dataframes.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file")

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head()


,Unnamed: 0,comment_id,product_id,content,customer_id,delivery_time,packaged_quality,seller_id,new_score,product_attributes,is_photo,thank_count,rating,date
0,0,19082694,247730209,"Camera cũng tạm, đủ sáng thì ok, camera trước ...",198202,"{'question': 'Thời gian giao hàng?', 'option':...","{'question': 'Cách đóng gói sản phẩm?', 'optio...",1.0,0.169220,Màu: Xanh,True,0,5,2023-04-12 15:24:16
1,1,19120490,247730209,"Mua lần nào cũng hài lòng , đặt hôm trước thì ...",8184126,NaN,NaN,1.0,0.168029,Màu: Xanh,True,2,5,2023-04-22 17:14:26
2,2,19211217,247730209,Nguyên seal,10717814,"{'question': 'Thời gian giao hàng?', 'option':...","{'question': 'Cách đóng gói sản phẩm?', 'optio...",1.0,0.145567,Màu: Đen,False,0,5,2023-05-19 21:17:59
3,3,19384790,247730209,NaN,5803037,NaN,NaN,1.0,0.145826,Màu: Xanh,False,0,5,2023-07-16 21:55:22
4,4,19154234,247730209,OK lem,29076404,NaN,NaN,1.0,0.144872,Màu: Đen,False,0,5,2023-05-03 17:49:10


In [3]:
combined_df.columns

Index(['Unnamed: 0', 'comment_id', 'product_id', 'content', 'customer_id',
       'delivery_time', 'packaged_quality', 'seller_id', 'new_score',
       'product_attributes', 'is_photo', 'thank_count', 'rating', 'date'],
      dtype='object')

In [4]:
combine = combined_df[['comment_id', 'product_id','content','customer_id','seller_id', 'rating', 'date']]
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199111 entries, 0 to 199110
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   comment_id   199111 non-null  int64  
 1   product_id   199111 non-null  int64  
 2   content      88491 non-null   object 
 3   customer_id  199111 non-null  int64  
 4   seller_id    199023 non-null  float64
 5   rating       199111 non-null  int64  
 6   date         195863 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 10.6+ MB


### 3. Data cleaning

In [5]:
combine = combine.dropna(subset='content')
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88491 entries, 0 to 199110
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comment_id   88491 non-null  int64  
 1   product_id   88491 non-null  int64  
 2   content      88491 non-null  object 
 3   customer_id  88491 non-null  int64  
 4   seller_id    88486 non-null  float64
 5   rating       88491 non-null  int64  
 6   date         87107 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 5.4+ MB


In [6]:
combine = combine.dropna(subset=['seller_id', 'date'])
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87107 entries, 0 to 199110
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comment_id   87107 non-null  int64  
 1   product_id   87107 non-null  int64  
 2   content      87107 non-null  object 
 3   customer_id  87107 non-null  int64  
 4   seller_id    87107 non-null  float64
 5   rating       87107 non-null  int64  
 6   date         87107 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 5.3+ MB


1. Loc comment do dai be hon 5 tu 
2. Chuan hoa thanh chu thuong 
3. Xoa cmt co emoji 
4. Xoa cmt co tu khong dau,... 
5. Xoa cmt co tu viet tat, nhay cam
6. Xoa stop word
7. Underthesea word tokenizer

In [7]:
combine["seller_id"] = combine["seller_id"].astype(int)
filtered_data = combine[combine["seller_id"] != 1]

In [8]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56762 entries, 18 to 199110
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   comment_id   56762 non-null  int64 
 1   product_id   56762 non-null  int64 
 2   content      56762 non-null  object
 3   customer_id  56762 non-null  int64 
 4   seller_id    56762 non-null  int32 
 5   rating       56762 non-null  int64 
 6   date         56762 non-null  object
dtypes: int32(1), int64(4), object(2)
memory usage: 3.2+ MB


In [9]:
def check_emoji(text):
    text_without_emoji = emoji.demojize(text)
    return text != text_without_emoji

filtered_data["has_emoji"] = filtered_data["content"].apply(check_emoji)
filtered_data = filtered_data[filtered_data['has_emoji'] == False]

filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55781 entries, 18 to 199110
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   comment_id   55781 non-null  int64 
 1   product_id   55781 non-null  int64 
 2   content      55781 non-null  object
 3   customer_id  55781 non-null  int64 
 4   seller_id    55781 non-null  int32 
 5   rating       55781 non-null  int64 
 6   date         55781 non-null  object
 7   has_emoji    55781 non-null  bool  
dtypes: bool(1), int32(1), int64(4), object(2)
memory usage: 3.2+ MB


C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\3833333149.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["has_emoji"] = filtered_data["content"].apply(check_emoji)


In [10]:
with open('teencode.txt',"r", encoding="utf-8") as file:
    teencode_words = [line.strip() for line in file]


def get_lower(text):
        return text.lower().strip()

filtered_data['content'] = filtered_data["content"].apply(get_lower)
filtered_data["word_count"] = filtered_data["content"].apply(lambda x: len(x.split()))
filtered_data["teencode_count"] = filtered_data["content"].apply(lambda x: sum(1 for word in x.split() if word in teencode_words))

filtered_data


,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng rất nhanh . hàng chính hãng đẹp như ...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
20,19303796,259542697,đẹp ạ,7489865,20704,5,2023-06-17 22:54:37,False,2,0
2734,18361436,203764665,máy mượt mới dùng đợi trải nghiệm,11111489,189668,5,2022-12-06 23:48:03,False,7,0
2735,19172478,203764665,"chơi game ổn, màn hình được và coi video chi t...",5451449,189668,5,2023-05-08 14:36:09,False,42,0
2736,18884049,203764665,nhanh chóng kỹ càng và máy rất hịn! cám ơn sho...,11136798,189668,5,2023-03-01 11:04:13,False,13,0
...,...,...,...,...,...,...,...,...,...,...
199106,7367825,56195676,đẹp,779643,41387,4,2021-01-30 09:35:20,False,1,0
199107,6008469,56195676,ok,1078859,41387,4,2020-12-07 16:43:43,False,1,0
199108,17935742,188832513,xuất hóa đơn hơi châm. dây thì oki,7440017,1989,3,2022-10-12 08:56:24,False,8,1
199109,16729140,48812108,hài lòng chất lượng,7508243,154143,5,2022-06-16 11:24:39,False,4,0


In [11]:
filtered_data['teencode_count'].value_counts()

0     50866
1      3950
2       682
3       193
4        50
5        18
6        10
7         6
8         2
9         2
12        1
11        1
Name: teencode_count, dtype: int64

In [12]:
filtered_data = filtered_data[(filtered_data["word_count"] > 7)& (filtered_data["word_count"] < 50) & (filtered_data["teencode_count"] < 2)]

In [13]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26577 entries, 18 to 199108
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment_id      26577 non-null  int64 
 1   product_id      26577 non-null  int64 
 2   content         26577 non-null  object
 3   customer_id     26577 non-null  int64 
 4   seller_id       26577 non-null  int32 
 5   rating          26577 non-null  int64 
 6   date            26577 non-null  object
 7   has_emoji       26577 non-null  bool  
 8   word_count      26577 non-null  int64 
 9   teencode_count  26577 non-null  int64 
dtypes: bool(1), int32(1), int64(6), object(2)
memory usage: 2.0+ MB


### 4. Data Exploring

In [14]:
def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')
    
    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )

with open('stopwords.txt', 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()

def remove_stopword(text):
    words = text.split()
    new_words = [word for word in words if word not in stopwords]
    return ' '.join(new_words)


def tokenize(text):
    return re.sub(r'\._+',' . ',word_tokenize(text, format="text"))

In [15]:
filtered_data['content'] = filtered_data['content'].apply(convert_unicode)

filtered_data

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\2486000120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(convert_unicode)


,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng rất nhanh . hàng chính hãng đẹp như ...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
2735,19172478,203764665,"chơi game ổn, màn hình được và coi video chi t...",5451449,189668,5,2023-05-08 14:36:09,False,42,0
2736,18884049,203764665,nhanh chóng kỹ càng và máy rất hịn! cám ơn sho...,11136798,189668,5,2023-03-01 11:04:13,False,13,0
2737,18442328,203764665,đã dùng pova 2. giờ mua thêm 4. tốt trong tầm giá,15720770,189668,5,2022-12-17 09:28:48,False,12,0
2738,18593332,203764665,"hàng rất đẹp, phụ kiện đầy đủ, máy cầm đầm tay...",21634978,189668,5,2023-01-08 01:01:13,False,22,0
...,...,...,...,...,...,...,...,...,...,...
199098,8744036,56195676,vừa mua về in đã bị kẹt giấy. mong phản hồi,7466585,41387,1,2021-03-29 12:49:28,False,11,0
199100,4317666,56195676,"đóng gói kỹ, máy nhìn ok, có điều ko cài in wi...",48346,41387,4,2020-08-04 09:35:04,False,19,0
199101,4565345,56195676,giao hàng đúng hẹn. sản phẩm hiện tại sử dụng tốt,16705719,41387,4,2020-08-22 09:22:59,False,11,0
199102,4928620,56195676,"ảo, scan ko có, có mỗi in và pho to, mực mờ",8345832,41387,1,2020-09-21 12:25:44,False,12,0


In [16]:
filtered_data['content'] = filtered_data['content'].apply(remove_stopword)
filtered_data

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\710103684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(remove_stopword)


,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng nhanh . hàng chính hãng đẹp giới thi...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
2735,19172478,203764665,"chơi game ổn, màn hình coi video chi tiết tông...",5451449,189668,5,2023-05-08 14:36:09,False,42,0
2736,18884049,203764665,nhanh chóng kỹ máy hịn! cám ơn shop tiki,11136798,189668,5,2023-03-01 11:04:13,False,13,0
2737,18442328,203764665,dùng pova 2. giờ mua thêm 4. tốt trong tầm giá,15720770,189668,5,2022-12-17 09:28:48,False,12,0
2738,18593332,203764665,"hàng đẹp, phụ kiện đầy đủ, máy cầm đầm tay chơ...",21634978,189668,5,2023-01-08 01:01:13,False,22,0
...,...,...,...,...,...,...,...,...,...,...
199098,8744036,56195676,mua về in kẹt giấy. mong phản hồi,7466585,41387,1,2021-03-29 12:49:28,False,11,0
199100,4317666,56195676,"đóng gói kỹ, máy nhìn ok, ko cài in wifi dc, k...",48346,41387,4,2020-08-04 09:35:04,False,19,0
199101,4565345,56195676,giao hàng đúng hẹn. sản phẩm hiện sử dụng tốt,16705719,41387,4,2020-08-22 09:22:59,False,11,0
199102,4928620,56195676,"ảo, scan ko có, in pho to, mực mờ",8345832,41387,1,2020-09-21 12:25:44,False,12,0


In [17]:
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)


def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True


def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)


def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = word.split('/')
        cw = [standardize_word_typing(w) for w in cw]

        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_punctuation(text):
    punctuation = '!"#$%&\'()*+-/:;<=>?@[\\]^`{|}~'
    translator = str.maketrans('', '', punctuation)
    return text.translate(translator)

In [18]:
def preprocess(text):
    text = text.strip()
    text = remove_punctuation(text)
    text = standardize_sentence_typing(text)
    return text 

filtered_data['content'] = filtered_data['content'].apply(preprocess)
filtered_data

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\36382727.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(preprocess)


,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng nhanh . hàng chính hãng đẹp giới thi...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
2735,19172478,203764665,"chơi game ổn, màn hình coi video chi tiết tông...",5451449,189668,5,2023-05-08 14:36:09,False,42,0
2736,18884049,203764665,nhanh chóng kỹ máy hịn cám ơn shop tiki,11136798,189668,5,2023-03-01 11:04:13,False,13,0
2737,18442328,203764665,dùng pova 2. giờ mua thêm 4. tốt trong tầm giá,15720770,189668,5,2022-12-17 09:28:48,False,12,0
2738,18593332,203764665,"hàng đẹp, phụ kiện đầy đủ, máy cầm đầm tay chơ...",21634978,189668,5,2023-01-08 01:01:13,False,22,0
...,...,...,...,...,...,...,...,...,...,...
199098,8744036,56195676,mua về in kẹt giâý. mong phản hồi,7466585,41387,1,2021-03-29 12:49:28,False,11,0
199100,4317666,56195676,"đóng gói kỹ, máy nhìn ok, ko cài in wifi dc, k...",48346,41387,4,2020-08-04 09:35:04,False,19,0
199101,4565345,56195676,giao hàng đúng hẹn. sản phẩm hiện sử dụng tốt,16705719,41387,4,2020-08-22 09:22:59,False,11,0
199102,4928620,56195676,"aỏ, scan ko có, in pho to, mực mờ",8345832,41387,1,2020-09-21 12:25:44,False,12,0


In [19]:
filtered_data['content'] = filtered_data['content'].apply(tokenize)
filtered_data

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\28645098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(tokenize)


,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng nhanh . hàng chính hãng đẹp giới_thi...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
2735,19172478,203764665,"chơi game_ổn , màn_hình coi video chi_tiết tôn...",5451449,189668,5,2023-05-08 14:36:09,False,42,0
2736,18884049,203764665,nhanh_chóng kỹ máy hịn cám_ơn shop tiki,11136798,189668,5,2023-03-01 11:04:13,False,13,0
2737,18442328,203764665,dùng pova 2 . giờ mua thêm 4 . tốt trong tầm giá,15720770,189668,5,2022-12-17 09:28:48,False,12,0
2738,18593332,203764665,"hàng đẹp , phụ_kiện đầy_đủ , máy cầm đầm tay_c...",21634978,189668,5,2023-01-08 01:01:13,False,22,0
...,...,...,...,...,...,...,...,...,...,...
199098,8744036,56195676,mua về in kẹt giâý . mong phản_hồi,7466585,41387,1,2021-03-29 12:49:28,False,11,0
199100,4317666,56195676,"đóng_gói kỹ , máy nhìn ok , ko cài in wifi dc ...",48346,41387,4,2020-08-04 09:35:04,False,19,0
199101,4565345,56195676,giao hàng đúng hẹn . sản_phẩm hiện sử_dụng tốt,16705719,41387,4,2020-08-22 09:22:59,False,11,0
199102,4928620,56195676,"ảo , scan ko có , in pho to , mực mờ",8345832,41387,1,2020-09-21 12:25:44,False,12,0


In [20]:
def has_numbers(text):
    return bool(re.search(r'\d', text))

# Loại bỏ các đánh giá có chứa số
filtered_data = filtered_data[~filtered_data['content'].apply(has_numbers)]
filtered_data

,comment_id,product_id,content,customer_id,seller_id,rating,date,has_emoji,word_count,teencode_count
18,19360301,259542697,giao hàng nhanh . hàng chính hãng đẹp giới_thi...,6303460,20704,5,2023-07-08 12:46:31,False,16,0
2736,18884049,203764665,nhanh_chóng kỹ máy hịn cám_ơn shop tiki,11136798,189668,5,2023-03-01 11:04:13,False,13,0
2738,18593332,203764665,"hàng đẹp , phụ_kiện đầy_đủ , máy cầm đầm tay_c...",21634978,189668,5,2023-01-08 01:01:13,False,22,0
2739,18540856,203764665,"quá tốt giá tiền . mua cha sử_dụng lướt_mượt ,...",290462,189668,5,2022-12-30 20:03:11,False,21,0
2744,18474709,203764665,dùng khá ổn tầm tiền bỏ ra . giáo hàng bọc cẩn...,8409991,189668,5,2022-12-21 21:58:58,False,17,0
...,...,...,...,...,...,...,...,...,...,...
199098,8744036,56195676,mua về in kẹt giâý . mong phản_hồi,7466585,41387,1,2021-03-29 12:49:28,False,11,0
199100,4317666,56195676,"đóng_gói kỹ , máy nhìn ok , ko cài in wifi dc ...",48346,41387,4,2020-08-04 09:35:04,False,19,0
199101,4565345,56195676,giao hàng đúng hẹn . sản_phẩm hiện sử_dụng tốt,16705719,41387,4,2020-08-22 09:22:59,False,11,0
199102,4928620,56195676,"ảo , scan ko có , in pho to , mực mờ",8345832,41387,1,2020-09-21 12:25:44,False,12,0


In [21]:
filtered_data['content'] = filtered_data['content'].apply(remove_stopword)
def remove_extra_dots(text):
    return re.sub(r'\.{2,}', '.', text)

filtered_data['content'] = filtered_data['content'].apply(remove_extra_dots)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\3672098209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(remove_stopword)
C:\Users\Rauciss\AppData\Local\Temp\ipykernel_6472\3672098209.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['content'] = filtered_data['content'].apply(remove_extra_dots)


In [22]:
result_df = filtered_data[['comment_id', 'product_id', 'content', 'customer_id', 'seller_id',
       'rating', 'date']]

result_df.to_csv('cleandata/tiki_comment.csv')